In [1]:
import pandas as pd

## Читає файл та змінює назви стовпців

In [2]:
df = pd.read_html('data/data.html')[0]
df.columns = ['Temp', 'L_value', 'R_value', 'A_M_value', 'Color_value', 'Spectral_Class_value', 'Type_value']

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temp                  261 non-null    int64  
 1   L_value               259 non-null    float64
 2   R_value               258 non-null    float64
 3   A_M_value             258 non-null    float64
 4   Color_value           260 non-null    object 
 5   Spectral_Class_value  259 non-null    object 
 6   Type_value            258 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 14.4+ KB


df.describe()

## Перевірка на наявність пропущених значень

In [4]:
print(df.isnull().sum())

Temp                    0
L_value                 2
R_value                 3
A_M_value               3
Color_value             1
Spectral_Class_value    2
Type_value              3
dtype: int64


## Заповнення пропущених значень

In [5]:
# Заповнення пропущених значень в числових стовпцях середнім значенням цих стовпців
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col].fillna(df[col].mean(), inplace=True)

# Заповнення пропущених значень в нечислових стовпцях модою (найчастіше зустрічається значення) цих стовпців
for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

## Перевірка аномальних значень

In [6]:
df_original = df.copy()
numeric_cols = df.select_dtypes(include=['float64', 'int64'])

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    # Визначення меж викидів
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Виявлення викидів
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"BEFORE: Outliers in {col}: {len(outliers)}")

    # Заміна значень викидів на медіану
    df.loc[df[col] < lower_bound, col] = df[col].median()
    df.loc[df[col] > upper_bound, col] = df[col].median()

    # Кількість викидів після заміни
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"AFTER: Outliers in {col}: {len(outliers)}\n")

# До / після
diff = df_original.compare(df)
print(f'\n{diff}')

BEFORE: Outliers in Temp: 9
AFTER: Outliers in Temp: 0

BEFORE: Outliers in L_value: 13
AFTER: Outliers in L_value: 0

BEFORE: Outliers in R_value: 47
AFTER: Outliers in R_value: 0

BEFORE: Outliers in A_M_value: 0
AFTER: Outliers in A_M_value: 0

BEFORE: Outliers in Type_value: 0
AFTER: Outliers in Type_value: 0


        Temp           L_value             R_value      
        self   other      self other          self other
30   39000.0  6850.0       NaN   NaN           NaN   NaN
50       NaN     NaN       NaN   NaN   1520.000000   1.8
51       NaN     NaN       NaN   NaN   1260.000000   1.8
52       NaN     NaN       NaN   NaN   1183.000000   1.8
53       NaN     NaN  550000.0  1.43   1648.000000   1.8
54       NaN     NaN       NaN   NaN   1324.000000   1.8
55       NaN     NaN       NaN   NaN   1349.000000   1.8
56       NaN     NaN       NaN   NaN   1673.000000   1.8
57       NaN     NaN       NaN   NaN   1284.000000   1.8
58       NaN     NaN       NaN   NaN    955.000000   1.8

## Cкільки разів кожне унікальне значення з'являється в стовпці

In [7]:
print(df['Color_value'].value_counts())
print('\n')
print(df['Spectral_Class_value'].value_counts())

Color_value
Red                    116
Blue                    67
Blue-white              30
Blue White              10
yellow-white             8
White                    7
Yellowish White          3
Orange                   3
white                    3
Blue white               3
Whitish                  2
yellowish                2
Pale yellow orange       1
What is the color?       1
White-Yellow             1
Yellowish                1
Something very DARK      1
Orange-Red               1
Blue-White               1
Name: count, dtype: int64


Spectral_Class_value
M        116
O         51
B         48
A         20
F         17
K          7
ERROR      1
G          1
Name: count, dtype: int64


## Стандартизація значень

In [8]:
# З попередньогу виводу можна помітити проблему, коли один і той же колір може бути представленний по різному:
# - Blue-white
# - Blue White
# - Blue-White
#
# Щоб вирішити цю проблему ми можемо привести всі значення до єдиного вигляду, тобто стандартизувати їх
# Приведемо всі кольори до вигляду: 'Color', 'Color-Color'
df['Color_value'] = df['Color_value'].str.title().str.replace(' ', '-')
print(df['Color_value'].value_counts())

Color_value
Red                    116
Blue                    67
Blue-White              44
White                   10
Yellow-White             8
Yellowish-White          3
Orange                   3
Yellowish                3
Whitish                  2
What-Is-The-Color?       1
Pale-Yellow-Orange       1
White-Yellow             1
Something-Very-Dark      1
Orange-Red               1
Name: count, dtype: int64


## Видалення рядків з помилковими категоріальними значеннями

In [9]:
# Трохи контексту
#
# Датафрейм містить інформацію про різні зорі. В данному випадку нас цікавлять стовбці з типом 'object',
# тобто 'Color_value' та 'Spectral_Class_value', які представляють колір зірки та її спектральний клас відповідно.
# Спектральний клас зірки визначається за допомогою системи класифікації Гарварда, яка базується на температурі зірки.
# Ця классифікація представляє собою розподіл зірок за діапазоном температур, де кожен клас представлений певною
# латинською літерою:
# - O: Найгарячіші зірки, з температурою від 30,000 до 60,000 Кельвінів. Вони випромінюють синє світло.
# - B: Гарячі зірки, з температурою від 10,000 до 30,000 Кельвінів. Вони випромінюють синьо-біле світло.
# - A: Зірки середньої температури, з температурою від 7,500 до 10,000 Кельвінів. Вони випромінюють біле світло.
# - F: Зірки з температурою від 6,000 до 7,500 Кельвінів. Вони випромінюють біло-жовте світло.
# - G: Зірки з температурою від 5,200 до 6,000 Кельвінів. Вони випромінюють жовте світло. Сонце належить до цього класу.
# - K: Зірки з температурою від 3,700 до 5,200 Кельвінів. Вони випромінюють помаранчеве світло.
# - M: Найхолодніші зірки, з температурою менше 3,700 Кельвінів. Вони випромінюють червоне світло.
#
# Таким чином ми маємо чіткий перелік допустимих значень для стовбця 'Spectral_Class_value' та можемо приблизно розуміти
# які кольори вважати допустимими.
df_original = df.copy()

valid_spectral_classes = ['O', 'B', 'A', 'F', 'G', 'K', 'M']
valid_colors = [
    'Blue',
    'Blue-White',
    'White',
    'Whitish',
    'Yellow-White',
    'Yellowish-White',
    'White-Yellow',
    'Yellow',
    'Yellowish',
    'Orange',
    'Orange-Red',
    'Red',
]

invalid_color_mask = ~df_original['Color_value'].isin(valid_colors)
invalid_spectral_class_mask = ~df_original['Spectral_Class_value'].isin(valid_spectral_classes)

invalid_rows = df_original[invalid_color_mask | invalid_spectral_class_mask]
print(f'Invalid rows:\n{invalid_rows}')

valid_rows_mask = df['Color_value'].isin(valid_colors) & df['Spectral_Class_value'].isin(valid_spectral_classes)
df = df[valid_rows_mask]

# Звісно можна піти далі і перевіряти чи відовідає фактична температура зірки діапазону температур
# її спектрального классу (і теж саме з кольором), але я не знаю наскільки це виправдано в даному випадку.

Invalid rows:
    Temp      L_value  R_value  A_M_value          Color_value  \
26  8570      0.00081   0.0097  14.200000   What-Is-The-Color?   
29  7230      0.00008   0.0130  14.080000   Pale-Yellow-Orange   
96  6850  55222.00000   1.8000   3.769922  Something-Very-Dark   

   Spectral_Class_value  Type_value  
26                    A    2.000000  
29                    F    2.000000  
96                ERROR    2.596899  


## Перевірка на наявність дублікатів та їх видалення

In [10]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)

19
